In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
import numpy as np
 
#Load Data with pandas, and parse the first column into datetime
data=pd.read_csv('data/alldistricts.csv')
data.head()

/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

,Unnamed: 0,DATE,CCN,STATION,NAME,AWND,PRCP,SNOW,SNWD,TMAX,TMIN,Day,Date,Month,Year,Illum
0,0,2008-01-01,66,USW00013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",13.42,0.0,0.0,0.0,54.0,36.0,3,1,1,2008,0.42
1,1,2008-01-02,66,USW00013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",18.34,0.0,0.0,0.0,38.0,25.0,6,1,2,2008,0.32
2,2,2008-01-03,75,USW00013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",10.74,0.0,0.0,0.0,34.0,22.0,7,1,3,2008,0.40
3,3,2008-01-04,65,USW00013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",5.82,0.0,0.0,0.0,41.0,21.0,3,1,4,2008,0.28
4,4,2008-01-05,88,USW00013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",3.58,0.0,0.0,0.0,43.0,28.0,5,1,5,2008,0.24


In [2]:
data.drop(data.columns[[0]], axis=1, inplace=True)
data.head()

,DATE,CCN,STATION,NAME,AWND,PRCP,SNOW,SNWD,TMAX,TMIN,Day,Date,Month,Year,Illum
0,2008-01-01,66,USW00013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",13.42,0.0,0.0,0.0,54.0,36.0,3,1,1,2008,0.42
1,2008-01-02,66,USW00013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",18.34,0.0,0.0,0.0,38.0,25.0,6,1,2,2008,0.32
2,2008-01-03,75,USW00013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",10.74,0.0,0.0,0.0,34.0,22.0,7,1,3,2008,0.40
3,2008-01-04,65,USW00013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",5.82,0.0,0.0,0.0,41.0,21.0,3,1,4,2008,0.28
4,2008-01-05,88,USW00013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",3.58,0.0,0.0,0.0,43.0,28.0,5,1,5,2008,0.24


In [3]:
data.rename(columns={"CCN": "CRIME-COUNT"}, inplace=True)

In [4]:
data['bins'] = np.nan

In [5]:
data.head()

,DATE,CRIME-COUNT,STATION,NAME,AWND,PRCP,SNOW,SNWD,TMAX,TMIN,Day,Date,Month,Year,Illum,bins
0,2008-01-01,66,USW00013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",13.42,0.0,0.0,0.0,54.0,36.0,3,1,1,2008,0.42,NaN
1,2008-01-02,66,USW00013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",18.34,0.0,0.0,0.0,38.0,25.0,6,1,2,2008,0.32,NaN
2,2008-01-03,75,USW00013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",10.74,0.0,0.0,0.0,34.0,22.0,7,1,3,2008,0.40,NaN
3,2008-01-04,65,USW00013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",5.82,0.0,0.0,0.0,41.0,21.0,3,1,4,2008,0.28,NaN
4,2008-01-05,88,USW00013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",3.58,0.0,0.0,0.0,43.0,28.0,5,1,5,2008,0.24,NaN


In [6]:
data['CRIME-COUNT'].describe()

count    4200.000000
mean       94.433810
std        18.374149
min        16.000000
25%        82.000000
50%        94.000000
75%       106.000000
max       163.000000
Name: CRIME-COUNT, dtype: float64

In [7]:
data.loc[data['CRIME-COUNT']<=62, ['bins']] = 'VeryLow'


In [8]:
data.loc[(data['CRIME-COUNT']>=63) & (data['CRIME-COUNT']<=83), ['bins']] = 'Low'

In [9]:
data.loc[(data['CRIME-COUNT']>=84) & (data['CRIME-COUNT']<=104), ['bins']] = 'Medium'

In [10]:
data.loc[(data['CRIME-COUNT']>=105) & (data['CRIME-COUNT']<=125), ['bins']] = 'High'

In [11]:
data.loc[(data['CRIME-COUNT']>=126, ['bins'])] = 'VeryHigh'

In [12]:
# data.loc[data['CRIME-COUNT']<=80, ['bins']] = 'Low'

In [13]:
# data.loc[(data['CRIME-COUNT']>=81) & (data['CRIME-COUNT']<=100), ['bins']] = 'Medium'

In [14]:
# data.loc[data['CRIME-COUNT']>=101, ['bins']] = 'High'

In [15]:
data['bins'].value_counts()

Medium      1830
Low         1014
High        1004
VeryHigh     208
VeryLow      144
Name: bins, dtype: int64

In [16]:
data.isnull().any()

DATE           False
CRIME-COUNT    False
STATION        False
NAME           False
AWND           False
PRCP           False
SNOW           False
SNWD           False
TMAX           False
TMIN           False
Day            False
Date           False
Month          False
Year           False
Illum          False
bins           False
dtype: bool

In [17]:
data.head()

,DATE,CRIME-COUNT,STATION,NAME,AWND,PRCP,SNOW,SNWD,TMAX,TMIN,Day,Date,Month,Year,Illum,bins
0,2008-01-01,66,USW00013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",13.42,0.0,0.0,0.0,54.0,36.0,3,1,1,2008,0.42,Low
1,2008-01-02,66,USW00013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",18.34,0.0,0.0,0.0,38.0,25.0,6,1,2,2008,0.32,Low
2,2008-01-03,75,USW00013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",10.74,0.0,0.0,0.0,34.0,22.0,7,1,3,2008,0.40,Low
3,2008-01-04,65,USW00013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",5.82,0.0,0.0,0.0,41.0,21.0,3,1,4,2008,0.28,Low
4,2008-01-05,88,USW00013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",3.58,0.0,0.0,0.0,43.0,28.0,5,1,5,2008,0.24,Medium


In [18]:
X = data.drop(["bins", "DATE", "STATION", "NAME", "CRIME-COUNT", "Year"], axis=1)
y = data["bins"]

In [19]:
data.head()

,DATE,CRIME-COUNT,STATION,NAME,AWND,PRCP,SNOW,SNWD,TMAX,TMIN,Day,Date,Month,Year,Illum,bins
0,2008-01-01,66,USW00013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",13.42,0.0,0.0,0.0,54.0,36.0,3,1,1,2008,0.42,Low
1,2008-01-02,66,USW00013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",18.34,0.0,0.0,0.0,38.0,25.0,6,1,2,2008,0.32,Low
2,2008-01-03,75,USW00013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",10.74,0.0,0.0,0.0,34.0,22.0,7,1,3,2008,0.40,Low
3,2008-01-04,65,USW00013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",5.82,0.0,0.0,0.0,41.0,21.0,3,1,4,2008,0.28,Low
4,2008-01-05,88,USW00013743,"WASHINGTON REAGAN NATIONAL AIRPORT, VA US",3.58,0.0,0.0,0.0,43.0,28.0,5,1,5,2008,0.24,Medium


In [20]:
# X['TMAX'] = X['TMAX'].astype(float)
# X['TMIN'] = X['TMIN'].astype(float)
# X['Month'] = X['Month'].astype(float)
# X['Date'] = X['Date'].astype(float)
# X['Day'] = X['Illum'].astype(float)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [22]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [23]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [24]:
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [26]:
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=10))
model.add(Dropout(.1))
model.add(Dense(units=100, activation='sigmoid'))
model.add(Dense(units=100, activation='tanh'))
model.add(Dropout(.1))
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(.1))
model.add(Dense(units=100, activation='tanh'))
model.add(Dropout(.1))
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(.1))
model.add(Dense(units=5, activation='softmax'))

W0822 14:37:07.872491 4600911296 deprecation.py:506] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [27]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               1100      
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0

In [29]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

Epoch 1/100
3150/3150 - 0s - loss: 1.3400 - acc: 0.4165
Epoch 2/100
3150/3150 - 0s - loss: 1.2867 - acc: 0.4375
Epoch 3/100
3150/3150 - 0s - loss: 1.2156 - acc: 0.4362
Epoch 4/100
3150/3150 - 0s - loss: 1.1922 - acc: 0.4559
Epoch 5/100
3150/3150 - 0s - loss: 1.1680 - acc: 0.4610
Epoch 6/100
3150/3150 - 0s - loss: 1.1711 - acc: 0.4705
Epoch 7/100
3150/3150 - 0s - loss: 1.1693 - acc: 0.4556
Epoch 8/100
3150/3150 - 0s - loss: 1.1586 - acc: 0.4737
Epoch 9/100
3150/3150 - 0s - loss: 1.1600 - acc: 0.4702
Epoch 10/100
3150/3150 - 0s - loss: 1.1654 - acc: 0.4632
Epoch 11/100
3150/3150 - 0s - loss: 1.1510 - acc: 0.4737
Epoch 12/100
3150/3150 - 0s - loss: 1.1505 - acc: 0.4644
Epoch 13/100
3150/3150 - 0s - loss: 1.1523 - acc: 0.4705
Epoch 14/100
3150/3150 - 0s - loss: 1.1447 - acc: 0.4717
Epoch 15/100
3150/3150 - 0s - loss: 1.1527 - acc: 0.4832
Epoch 16/100
3150/3150 - 0s - loss: 1.1425 - acc: 0.4844
Epoch 17/100
3150/3150 - 0s - loss: 1.1419 - acc: 0.4895
Epoch 18/100
3150/3150 - 0s - loss: 1.13

In [30]:
# X_train_scaled

In [31]:
# y_train_categorical.shape

In [32]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

1050/1050 - 0s - loss: 1.1404 - acc: 0.4705
Normal Neural Network - Loss: 1.1403549975440617, Accuracy: 0.4704761803150177


In [33]:
encoded_predictions = model.predict_classes(X_test_scaled[:30])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [34]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:30])}")

Predicted classes: ['High' 'Medium' 'Low' 'High' 'Low' 'Low' 'High' 'High' 'High' 'Medium'
 'Medium' 'Medium' 'Medium' 'Medium' 'Medium' 'High' 'Medium' 'High'
 'High' 'Medium' 'Medium' 'Medium' 'Medium' 'High' 'High' 'High' 'Low'
 'High' 'Medium' 'Low']
Actual Labels: ['Medium', 'Medium', 'Low', 'VeryHigh', 'Medium', 'Low', 'Medium', 'Medium', 'VeryHigh', 'High', 'High', 'Medium', 'High', 'High', 'Medium', 'Medium', 'Low', 'Low', 'High', 'Medium', 'Medium', 'High', 'High', 'High', 'Medium', 'Low', 'Medium', 'High', 'Medium', 'Medium']


In [35]:
model.save("static/models/alldistricts.h5")

In [36]:
model.save_weights("static/models/alldistricts_weights.h5")

In [37]:
with open('static/models/model_architecture_alldistricts.json', 'w') as f:
    f.write(model.to_json())

In [40]:
from tensorflow.keras.models import load_model
district1_model = load_model("static/models/district1.h5")

W0822 15:04:28.886652 4600911296 deprecation.py:506] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0822 15:04:28.893593 4600911296 deprecation.py:506] From /anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [42]:
district1_model.save_weights("static/models/district1_weights.h5")
with open('static/models/model_architecture_district1.json', 'w') as f:
    f.write(district1_model.to_json())

In [43]:
district2_model = load_model("static/models/district2.h5")
district2_model.save_weights("static/models/district2_weights.h5")
with open('static/models/model_architecture_district2.json', 'w') as f:
    f.write(district2_model.to_json())

In [44]:
district3_model = load_model("static/models/district3.h5")
district3_model.save_weights("static/models/district3_weights.h5")
with open('static/models/model_architecture_district3.json', 'w') as f:
    f.write(district3_model.to_json())

In [45]:
district4_model = load_model("static/models/district4.h5")
district4_model.save_weights("static/models/district4_weights.h5")
with open('static/models/model_architecture_district4.json', 'w') as f:
    f.write(district4_model.to_json())

In [46]:
district5_model = load_model("static/models/district5.h5")
district5_model.save_weights("static/models/district5_weights.h5")
with open('static/models/model_architecture_district5.json', 'w') as f:
    f.write(district5_model.to_json())

In [47]:
district6_model = load_model("static/models/district6.h5")
district6_model.save_weights("static/models/district6_weights.h5")
with open('static/models/model_architecture_district6.json', 'w') as f:
    f.write(district6_model.to_json())

In [48]:
district7_model = load_model("static/models/district7.h5")
district7_model.save_weights("static/models/district7_weights.h5")
with open('static/models/model_architecture_district7.json', 'w') as f:
    f.write(district7_model.to_json())